In [ ]:
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth', 10000)
pd.options.mode.chained_assignment = None
df_dev = pd.read_csv("../data/MedQA_dev.csv", index_col=0)
df_train = pd.read_csv("../data/MedQA_train.csv", index_col=0)


In [ ]:
#using Azure endpoint:
import openai
import os

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
from tech_train_functions import call_open_ai
client = None

Let's focus on question 7 and call it the hard study case:

In [ ]:
hard_study_case = df_dev.iloc[9]
pd.DataFrame(hard_study_case)

Let's try using BM25 search to find the most similar question to this in the test split.
First let's create BM25 search DB:

In [ ]:
#creating BM25 search DB:
from rank_bm25 import BM25Okapi
import numpy as np

questions_corpus=df_train['question'].tolist()
tokenized_question_corpus = [doc.split(" ") for doc in questions_corpus]
bm25_questions = BM25Okapi(tokenized_question_corpus)

def search_similar(query, bm25_index):
    query=query.lower()
    tokenized_query = query.split(" ")
    doc_scores = bm25_index.get_scores(tokenized_query)
    index=np.argsort(-doc_scores)[:2]
    return df_train.iloc[index]


In [ ]:
import os
from tech_train_functions import analyze_TA4H
ta4h_endpoint=os.getenv("TA4H_ENDPOINT")
key=os.getenv("TA4H_KEY")



Done! let's search for closest question to our example:

In [ ]:
most_similar_in_question = search_similar(hard_study_case["question"], bm25_questions)
pd.DataFrame(most_similar_in_question)

What if we used the answer options only for the search db?

In [ ]:
options_corpus=df_train['options'].astype(str).tolist()
tokenized_options_corpus = [doc.split(" ") for doc in options_corpus]
bm25_options = BM25Okapi(tokenized_options_corpus)

In [ ]:
most_similar_in_answer = search_similar(str(hard_study_case["options"]), bm25_options)
pd.DataFrame(most_similar_in_answer)

What if we used search only by relevant entities?

In [ ]:

hard_case_entities = analyze_TA4H(ta4h_endpoint, key, [hard_study_case["question"]])[0]
hard_case_entities = [e["text"] for e in hard_case_entities if e["category"] in ["SymptomOrSign","Diagnosis"] ]
hard_case_entities

In [ ]:
most_similar_in_entities = search_similar(" ".join(hard_case_entities), bm25_questions)
pd.DataFrame(most_similar_in_entities)

Let's see if these examples are better than a random one!
First let's create the CoT example: